In [8]:
import chemprop_ish as C
import torch
import pandas as pd
import torch.nn as nn

In [9]:
#TESTING SINGLE MPNN

args = C.TrainArgs()
args.atom_messages = False
model = C.MPNN(args)
out = model(['CC(=O)C','C','O'])
print(out)

data = pd.read_csv('bordwell_pka.csv')
data.columns = ['SMILES','pka']
SMILES = data['SMILES'].tolist()
pka = data['pka'].tolist()
pka = [torch.Tensor([x]) for x in pka]

optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
loss_function = nn.MSELoss()
for epoch in range(0, 5):
    for x in range(len(SMILES)):
        mol = [SMILES[x]]
        target = pka[x]
        optimizer.zero_grad()
        output = model(mol)[0]
        loss = loss_function(output, target)
        loss.backward()
        optimizer.step()
    print('Epoch:', epoch)
    print('Loss:', loss)

tensor([[0.0839],
        [0.0198],
        [0.0560]], grad_fn=<AddmmBackward>)
Epoch: 0
Loss: tensor(49.6656, grad_fn=<MseLossBackward>)
Epoch: 1
Loss: tensor(50.8828, grad_fn=<MseLossBackward>)
Epoch: 2
Loss: tensor(56.8960, grad_fn=<MseLossBackward>)
Epoch: 3
Loss: tensor(64.9495, grad_fn=<MseLossBackward>)
Epoch: 4
Loss: tensor(70.9275, grad_fn=<MseLossBackward>)


In [10]:
#TESTING DOUBLE MPNN

args = C.TrainArgs()
data = pd.read_csv('ETMdata.csv')
solute = data['solute'].tolist()
solvent = data['solvent'].tolist()
pka = data['pka'].tolist()
X_Y = [(x,y) for x,y in zip(solute,solvent)]
pka = [torch.Tensor([x]) for x in pka]
dmodel = C.double_MPNN(args)

optimizer = torch.optim.Adam(dmodel.parameters(), lr=1e-4)
loss_function = nn.MSELoss()
for epoch in range(0, 20):
    for x in range(len(X_Y)):
        mols = [X_Y[x]]
        target = pka[x]
        optimizer.zero_grad()
        output = dmodel(mols)[0]
        loss = loss_function(output, target)
        loss.backward()
        optimizer.step()
    print('Epoch:', epoch)
    print('Loss:', loss.item())

Epoch: 0
Loss: 147.4577178955078
Epoch: 1
Loss: 1.0285344123840332
Epoch: 2
Loss: 0.03546340391039848
Epoch: 3
Loss: 0.005249371286481619
Epoch: 4
Loss: 0.06924451887607574
Epoch: 5
Loss: 1.0274475812911987
Epoch: 6
Loss: 0.24772685766220093
Epoch: 7
Loss: 0.3669643700122833
Epoch: 8
Loss: 1.0308297872543335
Epoch: 9
Loss: 0.31833070516586304
Epoch: 10
Loss: 1.7746273279190063
Epoch: 11
Loss: 1.2564406394958496
Epoch: 12
Loss: 10.210241317749023
Epoch: 13
Loss: 5.803338050842285
Epoch: 14
Loss: 2.0440008640289307
Epoch: 15
Loss: 9.229966163635254
Epoch: 16
Loss: 5.338027477264404
Epoch: 17
Loss: 13.310317039489746
Epoch: 18
Loss: 8.062491416931152
Epoch: 19
Loss: 15.149951934814453


In [29]:
#TESTING DOUBLE MPNN WITH INTERACTION (CIGIN)

args = C.TrainArgs()
args.interaction = True

data = pd.read_csv('ETMdata.csv')
solute = data['solute'].tolist()
solvent = data['solvent'].tolist()
pka = data['pka'].tolist()
X_Y = [(x,y) for x,y in zip(solute,solvent)]
pka = [torch.Tensor([x]) for x in pka]
dmodel = C.double_MPNN(args)

In [30]:
print(dmodel(X_Y[0:5]))

tensor([[0.],
        [0.],
        [0.],
        [0.],
        [0.]], grad_fn=<ReluBackward0>)


In [15]:
optimizer = torch.optim.Adam(dmodel.parameters(), lr=1e-4)
loss_function = torch.nn.MSELoss()
for epoch in range(0, 20):
    for x in range(len(X_Y)):
        mols = [X_Y[x]]
        target = pka[x]
        optimizer.zero_grad()
        output = dmodel(mols)[0]
        loss = loss_function(output, target)
        loss.backward()
        optimizer.step()
    print('Epoch:', epoch)
    print('Loss:', loss.item())

Epoch: 0
Loss: 0.7457123398780823
Epoch: 1
Loss: 19.77728843688965
Epoch: 2
Loss: 17.387468338012695
Epoch: 3
Loss: 21.935728073120117
Epoch: 4
Loss: 19.714401245117188
Epoch: 5
Loss: 13.222643852233887
Epoch: 6
Loss: 14.366247177124023
Epoch: 7
Loss: 8.343389511108398
Epoch: 8
Loss: 14.248923301696777
Epoch: 9
Loss: 9.019465446472168
Epoch: 10
Loss: 16.309341430664062
Epoch: 11
Loss: 19.873170852661133
Epoch: 12
Loss: 18.615921020507812
Epoch: 13
Loss: 8.89854907989502
Epoch: 14
Loss: 23.737348556518555
Epoch: 15
Loss: 22.30984115600586
Epoch: 16
Loss: 18.19049644470215
Epoch: 17
Loss: 21.16509246826172
Epoch: 18
Loss: 6.100868225097656
Epoch: 19
Loss: 14.546182632446289
